In [38]:
import pandas as pd
from sklearn.compose import ColumnTransformer

# from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, RandomForestClassifier
from cuml.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error as rmse, r2_score
from sklearn.tree import export_graphviz, plot_tree
import matplotlib as plt


data = pd.read_csv("AmesHousing.csv")

column_names = data.columns
categorical_cols = list(data.select_dtypes(include=["object"]).columns)
numerical_cols = list(data.select_dtypes(include=["number"]).columns)


transformer = ColumnTransformer(
    transformers=[
        (
            "impute_means",
            SimpleImputer(missing_values=pd.NA, strategy="mean"),
            numerical_cols,
        ),
        (
            "impute_most_freq",
            SimpleImputer(missing_values=pd.NA, strategy="most_frequent"),
            categorical_cols,
        ),
    ],
    remainder="passthrough",
)

num_data = data.select_dtypes(exclude=object).fillna(
    data.select_dtypes(exclude=object).mean()
)
cat_data = data.select_dtypes(include=object).fillna(
    data.select_dtypes(include=object).mode().iloc[0]
)
data = pd.concat([num_data, cat_data], axis=1)
data = pd.get_dummies(data).astype(float)

In [2]:
X = data.drop("SalePrice", axis=1)
y = data["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
data.SalePrice.describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: SalePrice, dtype: float64

# random forest

overfit = model fit so close to data that it can't make good predictions. it won't be good for unseen data

we want to reduce the SSE

we can prune the trees to find an optimal subtree

we add a bias to the model so we don't have to prune the tree

cost complexity parameter minimize{SSE + a|T|} T = optimal nodes, a = cost complexity parameter. 

we want a large a so that we can have a smaller tree

a|T| = bias

## str

interpretable

fast predictions

important variables

## weaknesses

high variance in the predicted values due to overfitting

poor prediction accuracy








rmse = how off your prediction is on average

In [33]:
rf = RandomForestRegressor(
    n_estimators=600,
    split_criterion="mse",
    bootstrap=True,
    n_bins=10,
    max_leaves=-1,
    accuracy_metric="mse",
    min_samples_leaf=10
).fit(X_train, y_train)

/home/yogurtsauce/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


In [50]:
parameters = {
    'min_samples_split': [20, 25, 30],
    'min_samples_leaf': [5, 8, 10],
    'max_features': [2, 3, 4, 5, 6,7,8,9,10, 100, 200, 300],
}

In [51]:
grid = GridSearchCV(rf, parameters).fit(X_train, y_train)

/home/yogurtsauce/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


In [52]:
grid.best_estimator_

RandomForestRegressor()

In [54]:
grid.best_params_

{'max_features': 2, 'min_samples_leaf': 10, 'min_samples_split': 25}

In [53]:
grid.cv_results_

{'mean_fit_time': array([1.59179544, 1.54068675, 1.52969766, 1.41950469, 1.41826868,
        1.40548019, 1.36679993, 1.36635661, 1.3437005 , 1.71528478,
        1.69914956, 1.87792034, 1.60913539, 1.5901793 , 1.58976336,
        1.54542694, 1.5076077 , 1.50931854, 1.76933823, 1.88771305,
        1.84187961, 1.71235142, 1.6825264 , 1.68170829, 1.64793177,
        1.65758867, 1.64154124, 1.82278671, 1.80314841, 1.79699984,
        1.77385287, 1.77002506, 1.72310696, 1.70975571, 1.69612393,
        1.69731107, 1.87453246, 1.84850559, 1.79279494, 1.72198949,
        1.74785004, 1.76414838, 1.72412515, 1.72020831, 1.73197284,
        1.8409935 , 1.82839413, 1.83299632, 1.79775357, 1.76225877,
        1.76055884, 1.73051476, 1.72152529, 1.76008697, 1.85165491,
        1.82397594, 1.80227566, 1.7737433 , 1.76621876, 1.73983235,
        1.72909069, 1.79744477, 1.7474319 , 1.96826797, 1.82239861,
        1.79728751, 1.8176918 , 1.76835699, 1.76983008, 1.77976785,
        1.77498088, 1.76273637,

In [35]:
y_pred = rf.predict(X_test)
rmse(y_test, y_pred)

16723.71136440491

In [36]:
rf.score(X_test, y_test)

699072987.0899725

In [37]:
scores = cross_val_score(rf, X_test, y_test, cv=5, n_jobs=3)
score = scores.mean()
score

/home/yogurtsauce/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/home/yogurtsauce/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/home/yogurtsauce/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/home/yogurtsauce/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)
/home/yogurtsauce/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/cuml/internals/api_decor

941874940.180907

# bagging
bootstrap aggregating

bootstrap (take multiple samples with replacements)

you can make as many samples as you want

create m bootstrap samples from training data

train a single unpruned regression tree for each sample

average individual predictions from each tree

10 fold = 10 equal sized groups of the data

then we use 9 of them to test

it makes X_validation and y_validation